In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
data = pd.read_excel('../Stats_competition-/Teams_Data.xlsx')


In [3]:
data

,Team,Opponent,Team Score,Opponent Score,Team_TEAM,Team_CONF,Team_G,Team_ADJOE,Team_ADJDE,Team_BARTHAG,...,Opponent_ORB,Opponent_DRB,Opponent_FTR,Opponent_FTRD,Opponent_2P_O,Opponent_2P_D,Opponent_3P_O,Opponent_3P_D,Opponent_ADJ_T,Opponent_WAB
0,Duke,Maine,96,62,Duke,ACC,7,117.516,88.610,0.962610,...,23.5329,31.3239,22.7348,33.3174,51.3178,53.4241,29.3287,30.2950,67.5229,-2.2215
1,Duke,Army,100,58,Duke,ACC,7,117.516,88.610,0.962610,...,27.7235,32.5266,28.2289,23.4190,50.3197,52.8229,35.9850,33.3188,66.3286,-1.8179
2,Duke,Kentucky,72,77,Duke,ACC,7,117.516,88.610,0.962610,...,29.9181,22.8150,32.8196,24.3260,61.9600,45.7630,38.2360,24.9100,73.3500,1.3160
3,Duke,Wofford,86,35,Duke,ACC,7,117.516,88.610,0.962610,...,31.5135,34.5317,22.8347,36.8240,53.4125,54.6275,33.5161,36.4297,65.4317,-3.2300
4,Duke,Arizona,69,55,Duke,ACC,7,117.516,88.610,0.962610,...,40.1100,26.2580,34.0169,33.7182,53.9115,44.9460,31.0245,35.6266,71.9220,-1.4145
5,Duke,Kansas,72,75,Duke,ACC,7,117.516,88.610,0.962610,...,27.6239,23.5210,22.9345,32.3155,61.1900,47.0830,34.5125,28.7500,69.8900,2.2400
6,Duke,Seattle U,70,48,Duke,ACC,7,117.516,88.610,0.962610,...,31.5135,22.5110,45.1300,36.0221,43.8325,62.6358,34.1139,33.6196,67.0255,-3.2303
7,North Carolina,Elon,90,76,North Carolina,ACC,7,119.160,98.759,0.896518,...,35.6500,27.8105,38.7810,21.9110,49.5208,42.3200,35.7900,36.0286,66.6277,-1.3133
8,North Carolina,Kansas,89,92,North Carolina,ACC,7,119.160,98.759,0.896518,...,27.6239,23.5210,22.9345,32.3155,61.1900,47.0830,34.5125,28.7500,69.8900,2.2400
9,North Carolina,American University,107,55,North Carolina,ACC,7,119.160,98.759,0.896518,...,25.1295,31.5242,30.7238,42.1314,54.3108,53.5245,32.7188,34.5232,63.9355,-2.1210


In [4]:
data = data.dropna(subset=['Opponent_WAB'])

In [5]:
data.columns

Index(['Team', 'Opponent', 'Team Score', 'Opponent Score', 'Team_TEAM',
       'Team_CONF', 'Team_G', 'Team_ADJOE', 'Team_ADJDE', 'Team_BARTHAG',
       'Team_EFG_O', 'Team_EFG_D', 'Team_TOR', 'Team_TORD', 'Team_ORB',
       'Team_DRB', 'Team_FTR', 'Team_FTRD', 'Team_2P_O', 'Team_2P_D',
       'Team_3P_O', 'Team_3P_D', 'Team_ADJ_T', 'Team_WAB', 'Opponent_TEAM',
       'Opponent_CONF', 'Opponent_G', 'Opponent_ADJOE', 'Opponent_ADJDE',
       'Opponent_BARTHAG', 'Opponent_EFG_O', 'Opponent_EFG_D', 'Opponent_TOR',
       'Opponent_TORD', 'Opponent_ORB', 'Opponent_DRB', 'Opponent_FTR',
       'Opponent_FTRD', 'Opponent_2P_O', 'Opponent_2P_D', 'Opponent_3P_O',
       'Opponent_3P_D', 'Opponent_ADJ_T', 'Opponent_WAB'],
      dtype='object')

In [18]:
feature_cols = data.columns.difference(['Team', 'Opponent', 'Team Score', 'Opponent Score', 'Team_TEAM','Team_CONF', 'Team_G','Opponent_TEAM','Opponent_CONF', 'Opponent_G'])
X = data[feature_cols]
y = data[['Team Score', 'Opponent Score']]


for column in feature_cols:
    print(column)

Opponent_2P_D
Opponent_2P_O
Opponent_3P_D
Opponent_3P_O
Opponent_ADJDE
Opponent_ADJOE
Opponent_ADJ_T
Opponent_BARTHAG
Opponent_DRB
Opponent_EFG_D
Opponent_EFG_O
Opponent_FTR
Opponent_FTRD
Opponent_ORB
Opponent_TOR
Opponent_TORD
Opponent_WAB
Team_2P_D
Team_2P_O
Team_3P_D
Team_3P_O
Team_ADJDE
Team_ADJOE
Team_ADJ_T
Team_BARTHAG
Team_DRB
Team_EFG_D
Team_EFG_O
Team_FTR
Team_FTRD
Team_ORB
Team_TOR
Team_TORD
Team_WAB


In [7]:
def train_and_select_best_models(X, y):
    models = {}
    for target in y.columns:
        y_target = y[target]
        X_train, X_test, y_train, y_test = train_test_split(X, y_target, test_size=0.2, random_state=42)

        candidate_models = {
            'RandomForest': RandomForestRegressor(n_estimators=100, random_state=42),
            'GradientBoosting': GradientBoostingRegressor(random_state=42),
            'LinearRegression': LinearRegression()
        }

        best_model = None
        best_score = float('inf')
        for name, model in candidate_models.items():
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            rmse = mean_squared_error(y_test, y_pred, squared=False)
            print(f"{target} - {name}: RMSE = {rmse:.4f}")
            if rmse < best_score:
                best_score = rmse
                best_model = model

        models[target] = best_model
        
    return models

In [8]:
models = train_and_select_best_models(X, y)

Team Score - RandomForest: RMSE = 14.1992
Team Score - GradientBoosting: RMSE = 16.2793
Team Score - LinearRegression: RMSE = 10.1629
Opponent Score - RandomForest: RMSE = 7.5801
Opponent Score - GradientBoosting: RMSE = 14.8409
Opponent Score - LinearRegression: RMSE = 40.2279


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_reg

In [9]:
models

{'Team Score': LinearRegression(),
 'Opponent Score': RandomForestRegressor(random_state=42)}

In [10]:
def predict_and_simulate(models, X, n_simulations=10000):
    results = []

    for idx, row in X.iterrows():
        row_results = {'Index': idx}

        for target, model in models.items():
            score_pred = model.predict([row])[0]

            residual_std = 10 
            simulated_scores = np.random.normal(score_pred, residual_std, n_simulations)
            confidence_interval = (np.percentile(simulated_scores, 2.5), np.percentile(simulated_scores, 97.5))

            row_results[f'{target}_Predicted_Score'] = score_pred
            row_results[f'{target}_Confidence_Interval'] = confidence_interval

        results.append(row_results)

    return pd.DataFrame(results)

In [11]:
test_df = pd.read_excel('../Stats_competition-/Teams_Data_test.xlsx')

In [12]:
test_df

,Team,Opponent,Team_ADJOE,Team_ADJDE,Team_BARTHAG,Team_EFG_O,Team_EFG_D,Team_TOR,Team_TORD,Team_ORB,...,Opponent_ORB,Opponent_DRB,Opponent_FTR,Opponent_FTRD,Opponent_2P_O,Opponent_2P_D,Opponent_3P_O,Opponent_3P_D,Opponent_ADJ_T,Opponent_WAB
0,Duke,Auburn,117.516,88.610,0.962610,55.352,40.7500,16.612,21.3540,33.6940,...,32.3123,25.4430,29.6264,30.6121,65.7100,42.8250,36.3730,33.9211,68.9139,3.0100
1,Duke,Louisville,117.516,88.610,0.962610,55.352,40.7500,16.612,21.3540,33.6940,...,36.2410,30.9220,33.9174,33.2172,61.6800,43.9330,28.7298,33.1179,70.1780,0.1953
2,North Carolina,Alabama,119.160,98.759,0.896518,54.666,50.2171,12.740,14.4315,25.9279,...,35.6500,28.7140,46.3230,38.6269,60.3140,46.2690,32.5194,29.5700,73.2700,1.8600
3,North Carolina,Georgia Tech,119.160,98.759,0.896518,54.666,50.2171,12.740,14.4315,25.9279,...,32.5121,25.6460,31.5224,37.4249,51.4173,44.9460,32.6192,35.3255,70.8470,-1.9194
4,N.C. State,Texas,110.285,98.355,0.787168,52.312,45.1450,15.157,22.1330,29.5192,...,28.3216,27.1840,38.0910,28.7900,58.2340,38.2100,35.7900,33.6196,68.6160,0.3049
5,N.C. State,Florida State,110.285,98.355,0.787168,52.312,45.1450,15.157,22.1330,29.5192,...,29.9181,30.4201,45.5270,40.4296,54.7980,49.4142,32.5194,26.4170,72.1200,0.1854


In [13]:
test_X = test_df[feature_cols] 

In [14]:
test_X

,Opponent_2P_D,Opponent_2P_O,Opponent_3P_D,Opponent_3P_O,Opponent_ADJDE,Opponent_ADJOE,Opponent_ADJ_T,Opponent_BARTHAG,Opponent_DRB,Opponent_EFG_D,...,Team_BARTHAG,Team_DRB,Team_EFG_D,Team_EFG_O,Team_FTR,Team_FTRD,Team_ORB,Team_TOR,Team_TORD,Team_WAB
0,42.8250,65.7100,33.9211,36.3730,94.4110,123.7100,68.9139,0.957140,25.4430,45.0420,...,0.962610,24.0250,40.7500,55.352,26.7306,27.1620,33.6940,16.612,21.3540,0.4543
1,43.9330,61.6800,33.1179,28.7298,97.9500,113.0540,70.1780,0.838645,30.9220,46.2570,...,0.962610,24.0250,40.7500,55.352,26.7306,27.1620,33.6940,16.612,21.3540,0.4543
2,46.2690,60.3140,29.5700,32.5194,96.8330,121.7300,73.2700,0.932780,28.7140,45.5510,...,0.896518,28.0113,50.2171,54.666,43.2440,28.1810,25.9279,12.740,14.4315,0.0463
3,44.9460,51.4173,35.3255,32.6192,103.3127,109.6910,70.8470,0.664711,25.6460,47.8101,...,0.896518,28.0113,50.2171,54.666,43.2440,28.1810,25.9279,12.740,14.4315,0.0463
4,38.2100,58.2340,33.6196,35.7900,95.5190,115.0360,68.6160,0.894621,27.1840,42.6190,...,0.787168,33.5290,45.1450,52.312,37.4990,30.0109,29.5192,15.157,22.1330,-0.5940
5,49.4142,54.7980,26.4170,32.5194,96.0240,107.6121,72.1200,0.786870,30.4201,44.8370,...,0.787168,33.5290,45.1450,52.312,37.4990,30.0109,29.5192,15.157,22.1330,-0.5940


In [15]:
results_df = predict_and_simulate(models, test_X)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: 

In [16]:
df_combined = pd.concat([test_df, results_df.drop('Index', axis = 1)], axis = 1)

In [17]:
df_combined[['Team', 'Opponent','Team Score_Predicted_Score',
       'Team Score_Confidence_Interval', 'Opponent Score_Predicted_Score',
       'Opponent Score_Confidence_Interval']]

,Team,Opponent,Team Score_Predicted_Score,Team Score_Confidence_Interval,Opponent Score_Predicted_Score,Opponent Score_Confidence_Interval
0,Duke,Auburn,55.000000,"(34.91685095793404, 74.92282005266087)",78.66,"(59.06679532598899, 98.54622728980583)"
1,Duke,Louisville,96.802176,"(77.6317966931645, 116.55036736207876)",73.18,"(53.55548003061366, 92.96672537574457)"
2,North Carolina,Alabama,66.726569,"(47.23211913561412, 86.16607753231183)",77.60,"(58.03572935197132, 97.40019141328195)"
3,North Carolina,Georgia Tech,84.338049,"(64.95337503368968, 104.05695495403161)",65.73,"(46.426970097790175, 85.19339058920508)"
4,N.C. State,Texas,66.677328,"(47.15843546681768, 86.0620257920547)",79.15,"(59.990393637890925, 98.77533839630168)"
5,N.C. State,Florida State,82.207192,"(62.262513158288456, 101.84623967793308)",73.99,"(54.08900649039275, 93.24429869962462)"
